In [1]:
data_var = '2024-01-11'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9772,2024-01-11,Eua Ncaa,23:00,Montana State,Northern Arizona,1.73,2.18,143.5,1.80,1.95,-3.5,2.00,1.71,xS6ORyZi,0.578035,0.458716,0.555556,0.512821,0.036750,218.138,126.844776,0.581489,1.2,1.643168,1.369306,439.20,3.158,2.321674,0.735172,-37.0,217.170,206.485090,0.950799,1.8,1.643168,0.912871,122.10,3.746,4.341558,1.158985,-62.0,61,74,7.20,1.65,121.574,311.410,0.162761,0.056569,0.110545,-1.84,-0.368,-1.983696,0.0,0.0,0.0,0.09,0.018,65.555556,0.000000,0.0,0.000000
9773,2024-01-11,Eua Ncaa,23:00,Montana,Northern Colorado,1.30,3.66,156.5,1.91,1.91,-8.5,2.00,1.71,Wl6SQelb,0.769231,0.273224,0.523560,0.523560,0.042455,139.238,34.412697,0.247150,2.4,1.341641,0.559017,92.66,1.826,0.569588,0.311932,19.0,453.550,549.492084,1.211536,1.8,1.643168,0.912871,135.88,6.074,8.397710,1.382567,-6.0,82,86,1.13,1.58,0.000,465.110,0.672892,0.000000,0.110545,0.00,0.000,inf,0.0,0.0,0.0,0.88,0.176,15.113636,0.000000,0.0,0.000000
9774,2024-01-11,Eua Ncaa,23:00,Middle Tenn. St.,Louisiana Tech,3.29,1.37,127.5,1.91,1.91,5.5,2.00,1.71,f54jAG7G,0.303951,0.729927,0.523560,0.523560,0.033878,172.416,79.590850,0.461621,0.6,1.341641,2.236068,140.94,3.558,3.168110,0.890419,-70.0,143.046,56.044157,0.391791,0.6,1.341641,2.236068,107.80,1.920,0.891263,0.464200,15.0,54,77,2.61,1.40,108.734,148.798,0.582680,0.000000,0.110545,-1.55,-0.310,-7.387097,0.0,0.0,0.0,-2.77,-0.554,-0.667870,0.000000,0.0,0.000000
9775,2024-01-11,Eua Ncaa,23:00,Longwood,Radford,1.49,2.74,134.5,1.91,1.91,-6.5,2.10,1.67,l4sf8LL4,0.671141,0.364964,0.523560,0.523560,0.036104,175.048,181.080356,1.034461,1.2,1.643168,1.369306,81.62,2.566,2.625753,1.023287,14.0,215.596,188.030309,0.872142,1.8,1.643168,0.912871,127.80,3.422,3.385989,0.989477,-24.0,77,71,1.06,1.80,83.352,264.002,0.417912,0.000000,0.161303,0.45,0.090,5.444444,0.0,0.0,0.0,-0.07,-0.014,-124.285714,0.000000,0.0,0.000000
9776,2024-01-11,Eua Ncaa,23:00,Illinois,Michigan State,1.63,2.38,149.5,1.80,1.95,-4.5,2.00,1.71,r9Z018Wi,0.613497,0.420168,0.555556,0.512821,0.033665,185.012,123.649139,0.668330,1.2,1.643168,1.369306,381.42,2.326,1.595268,0.685842,11.0,103.400,14.875915,0.143868,2.4,1.341641,0.559017,125.06,1.218,0.276170,0.226741,90.0,78,74,4.89,1.69,127.100,0.000,0.264504,0.056569,0.110545,0.70,0.140,4.500000,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9855,2024-01-11,Eua Ncaa,20:30,UMass Lowell,NJIT,1.06,10.50,147.5,1.91,1.91,-15.5,2.10,1.67,CdDDWWI0,0.943396,0.095238,0.523560,0.523560,0.038634,116.314,44.970034,0.386626,1.8,1.643168,0.912871,159.90,1.576,0.467365,0.296551,11.0,541.680,506.656928,0.935344,1.2,1.643168,1.369306,173.74,7.900,8.084884,1.023403,-19.0,82,73,1.95,2.38,0.000,782.312,1.154860,0.000000,0.161303,0.00,0.000,inf,0.0,0.0,0.0,7.50,1.500,6.333333,0.000000,0.0,0.000000
9856,2024-01-11,Eua Ncaa,20:00,Maine,New Hampshire,1.69,2.18,143.5,1.83,1.83,-2.5,1.83,1.83,YB1MUAmD,0.591716,0.458716,0.546448,0.546448,0.050432,299.820,177.456066,0.591875,0.6,1.341641,2.236068,278.40,5.090,3.367150,0.661523,-43.0,145.800,50.958694,0.349511,1.2,1.643168,1.369306,140.25,2.002,0.697

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9778,22:45,Eua Ncaa,North Alabama,Bellarmine,1.55,Back Home
9782,22:30,Eua Ncaa,Louisiana Monroe,Southern Miss,2.47,Back Home
9792,17:15,Islândia Premier League,Grindavik,Alftanes,2.03,Back Home
9796,21:15,Uruguai Liga Uruguaia,Defensor Sporting,Bigua,1.70,Back Home
9814,22:00,Eua Ncaa,Nebraska O.,North Dakota,1.69,Back Home
9829,16:00,Eua Nba,Cleveland Cavaliers,Brooklyn Nets,1.69,Back Home
9847,21:00,Eua Ncaa,Hampton,William & Mary,1.87,Back Home
9858,22:00,Eua Ncaa,Alabama A&M,Alcorn State,1.91,Back Home
